# Unit 8 — Advanced OOP & Design Principles

**Purpose:** Improve maintainability and scalability.

This notebook is intentionally detailed:
- short theory blocks,
- many runnable examples,
- practice exercises with starter code.

Topics covered:
- Encapsulation and access control
- Private attributes and properties
- Abstract base classes (`abc`)
- Class methods and static methods
- Composition vs. inheritance
- Design patterns: Factory, Singleton

## Learning goals

By the end of this unit, you will be able to:

- Apply encapsulation to protect object invariants
- Use properties to validate and control attribute access
- Define interfaces/contracts with Abstract Base Classes
- Use `@classmethod` and `@staticmethod` appropriately
- Choose composition vs inheritance using "has-a" vs "is-a"
- Implement a simple Factory pattern
- Understand Singleton trade-offs and implement a safe version

---

# 1) Encapsulation and access control (Python style)

Python does not enforce access modifiers like Java/C#.
Instead, it uses conventions:

- `attr` → public API
- `_attr` → internal/protected by convention (do not use outside the class unless you must)
- `__attr` → name-mangled attribute (stronger signal; prevents accidental access)

Encapsulation is less about "hiding" and more about:
- keeping objects in valid states,
- providing stable public interfaces,
- preventing accidental misuse.

In [ ]:
class BankAccount:
    def __init__(self, owner, balance=0.0):
        self.owner = owner
        self._balance = float(balance)  # internal

acc = BankAccount("Alice", 100)
print(acc.owner)
print(acc._balance)  # accessible but discouraged

### Why encapsulate?

Suppose we want to enforce:
- balance cannot become negative
- deposits must be positive

If everyone can write `acc._balance = -999`, our object can become invalid.
We'll fix that using methods and properties.

In [ ]:
class SafeAccount:
    def __init__(self, owner, balance=0.0):
        self.owner = owner
        self._balance = float(balance)

    def deposit(self, amount):
        amount = float(amount)
        if amount <= 0:
            raise ValueError("deposit must be positive")
        self._balance += amount

    def withdraw(self, amount):
        amount = float(amount)
        if amount <= 0:
            raise ValueError("withdraw must be positive")
        if amount > self._balance:
            raise ValueError("insufficient funds")
        self._balance -= amount

    def balance(self):
        return self._balance

a = SafeAccount("Bob", 50)
a.deposit(10)
a.withdraw(20)
print(a.balance())

---

# 2) Private attributes (`__attr`) and name mangling

Double underscore triggers name mangling:
- `self.__balance` becomes `self._ClassName__balance`.

This is not true privacy, but it prevents accidental access.

In [ ]:
class Account:
    def __init__(self, owner, balance=0.0):
        self.owner = owner
        self.__balance = float(balance)

    def deposit(self, amount):
        amount = float(amount)
        if amount <= 0:
            raise ValueError("amount must be positive")
        self.__balance += amount

    def get_balance(self):
        return self.__balance

acc = Account("Alice", 100)
acc.deposit(50)
print("balance via method:", acc.get_balance())

# Uncomment to see AttributeError:
# print(acc.__balance)

print("mangled name access:", acc._Account__balance)  # possible but discouraged

---

# 3) Properties: controlled attribute access

Properties allow attribute-like access while using methods underneath.
This is the most Pythonic way to do encapsulation for simple attributes.

Typical use cases:
- validate values
- keep derived values consistent
- create read-only attributes

In [ ]:
class Temperature:
    def __init__(self, celsius):
        self.celsius = celsius  # uses the setter

    @property
    def celsius(self):
        return self._celsius

    @celsius.setter
    def celsius(self, value):
        value = float(value)
        if value < -273.15:
            raise ValueError("below absolute zero")
        self._celsius = value

    @property
    def fahrenheit(self):
        return self._celsius * 9/5 + 32

t = Temperature(20)
print(t.celsius, t.fahrenheit)
t.celsius = 25
print(t.celsius, t.fahrenheit)

### Read-only properties

If you define only a getter, the attribute cannot be assigned.

In [ ]:
class User:
    def __init__(self, username):
        self._username = username

    @property
    def username(self):
        return self._username

u = User("alice")
print(u.username)

# Uncomment to see AttributeError:
# u.username = "bob"

---

# 4) Abstract Base Classes (ABC)

Abstract base classes define a contract: what methods must exist.

Why this matters:
- You can swap implementations without changing calling code.
- It enables clean testing and modularity.

We will model a storage interface similar to our Unit 6 project.

In [ ]:
from abc import ABC, abstractmethod

class Storage(ABC):
    @abstractmethod
    def save(self, data):
        pass

    @abstractmethod
    def load(self):
        pass

### Two implementations: JSONStorage and CSVStorage

Both satisfy the same `Storage` interface.

In [ ]:
import json
import csv
from pathlib import Path

class JSONStorage(Storage):
    def __init__(self, path):
        self.path = Path(path)

    def save(self, data):
        self.path.parent.mkdir(parents=True, exist_ok=True)
        with self.path.open("w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

    def load(self):
        if not self.path.exists():
            return []
        with self.path.open("r", encoding="utf-8") as f:
            return json.load(f)

class CSVStorage(Storage):
    def __init__(self, path, fieldnames):
        self.path = Path(path)
        self.fieldnames = list(fieldnames)

    def save(self, data):
        self.path.parent.mkdir(parents=True, exist_ok=True)
        with self.path.open("w", newline="", encoding="utf-8") as f:
            w = csv.DictWriter(f, fieldnames=self.fieldnames)
            w.writeheader()
            for row in data:
                w.writerow(row)

    def load(self):
        if not self.path.exists():
            return []
        with self.path.open("r", newline="", encoding="utf-8") as f:
            r = csv.DictReader(f)
            return list(r)

sample = [{"name": "Alice", "age": 30}, {"name": "Bob", "age": 25}]
js = JSONStorage("data_u8/sample.json")
js.save(sample)
print("json:", js.load())

cs = CSVStorage("data_u8/sample.csv", ["name", "age"])
cs.save(sample)
print("csv:", cs.load())

### Depending on an interface, not an implementation

The function below can work with any `Storage` implementation.

In [ ]:
def store_and_reload(storage: Storage, data):
    storage.save(data)
    return storage.load()

print(store_and_reload(JSONStorage("data_u8/test.json"), [{"x": 1}]))
print(store_and_reload(CSVStorage("data_u8/test.csv", ["x"]), [{"x": 1}]))

---

# 5) Instance vs class vs static methods

- Instance method: `def m(self, ...)` → uses object state
- Class method: `@classmethod def m(cls, ...)` → uses class, often alternative constructors
- Static method: `@staticmethod def m(...)` → utility inside class namespace

We'll use an example with alternative constructors and validation helpers.

In [ ]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = int(age)

    @classmethod
    def from_string(cls, text):
        # Example input: "Alice,30"
        name, age = text.split(",")
        return cls(name.strip(), int(age.strip()))

    @staticmethod
    def is_adult(age):
        return int(age) >= 18

p = Person.from_string("Alice, 30")
print(p.name, p.age)
print(Person.is_adult(17), Person.is_adult(20))

---

# 6) Composition vs inheritance

Rule of thumb:
- Inheritance for "is-a"
- Composition for "has-a"

Prefer composition when you want flexibility: you can swap parts without changing the main class.

### Composition example: Report uses an exporter object

In [ ]:
class TextExporter:
    def export(self, title, data):
        lines = [f"REPORT: {title}", "-" * 20]
        for k, v in data.items():
            lines.append(f"{k}: {v}")
        return "\n".join(lines)

class JSONExporter:
    def export(self, title, data):
        return json.dumps({"title": title, "data": data}, ensure_ascii=False, indent=2)

class Report:
    def __init__(self, title, data, exporter):
        self.title = title
        self.data = data
        self.exporter = exporter  # composition

    def export(self):
        return self.exporter.export(self.title, self.data)

r = Report("Weekly", {"sales": 10, "profit": 3}, TextExporter())
print(r.export())
r.exporter = JSONExporter()
print(r.export())

### Inheritance example: SavingsAccount is a specialized Account

In [ ]:
class AccountBase:
    def __init__(self, owner, balance=0.0):
        self.owner = owner
        self.balance = float(balance)

    def deposit(self, amount):
        amount = float(amount)
        if amount <= 0:
            raise ValueError("amount must be positive")
        self.balance += amount

class SavingsAccount(AccountBase):
    def apply_interest(self, rate):
        rate = float(rate)
        if rate < 0:
            raise ValueError("rate must be >= 0")
        self.balance += self.balance * rate

sa = SavingsAccount("Alice", 1000)
sa.apply_interest(0.05)
print(sa.balance)

---

# 7) Design Pattern: Factory

A Factory centralizes object creation.
This is useful when:
- there are many subclasses,
- construction logic differs,
- you want to avoid scattered `if/elif` in code.

We'll build an Employee Factory.

In [ ]:
class Employee:
    def __init__(self, name):
        self.name = name

    def role(self):
        return "Employee"

class Developer(Employee):
    def role(self):
        return "Developer"

class Manager(Employee):
    def role(self):
        return "Manager"

class Intern(Employee):
    def role(self):
        return "Intern"

### Factory with registry (recommended)

In [ ]:
class EmployeeFactory:
    registry = {
        "employee": Employee,
        "developer": Developer,
        "manager": Manager,
        "intern": Intern,
    }

    @classmethod
    def create(cls, employee_type, name):
        key = employee_type.lower().strip()
        if key not in cls.registry:
            raise ValueError(f"Unknown employee type: {employee_type}")
        return cls.registry[key](name)

team = [
    EmployeeFactory.create("developer", "Alice"),
    EmployeeFactory.create("manager", "Bob"),
    EmployeeFactory.create("intern", "Charlie"),
]
for e in team:
    print(e.name, "->", e.role())

---

# 8) Design Pattern: Singleton

A Singleton ensures only one instance exists.

Caution:
- It introduces global state and can make testing harder.
- Many Python projects prefer module-level singletons instead.

We'll implement it to recognize the pattern.

In [ ]:
class Config:
    _instance = None

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
            cls._instance._initialized = False
        return cls._instance

    def __init__(self, settings=None):
        if self._initialized:
            return
        self.settings = dict(settings or {})
        self._initialized = True

c1 = Config({"env": "dev"})
c2 = Config({"env": "prod"})

print("Same object?", c1 is c2)
print("Settings:", c1.settings)

### Preferred alternative: module-level config object

In practice, a module file like `config.py` can contain:
```python
SETTINGS = {...}
```
and you simply import it where needed.

This is simpler and more testable than a Singleton class.

---

# Practice — Build a small system (Employee Management + Factory)

This practice ties together most of Unit 8.
You will implement:
- encapsulation,
- properties,
- a factory,
- class methods,
- basic reporting.

(Optionally, you can add an ABC-based interface for exporters or storage.)

## Exercise 1 — Employee system + Company + Factory (recommended)

**Requirements**
- `Employee` base class with `name`
- subclasses: `Developer`, `Manager`, `Intern`
- property `email` generated from name (e.g. `alice.smith@company.com`)
- `EmployeeFactory` registry-based creation by string type
- `Company` stores employees in `_employees` (encapsulation) and supports:
  - `add(employee)`
  - `list_by_role(role)`
  - `headcount_by_role() -> dict`

Implement and run the demo.

In [ ]:
# Exercise 1 starter

class Employee2:
    def __init__(self, name):
        self.name = name

    @property
    def email(self):
        # TODO: generate from name
        # Suggestion: lowercase, spaces -> dots
        pass

    def role(self):
        return "Employee"

class Developer2(Employee2):
    def role(self):
        return "Developer"

class Manager2(Employee2):
    def role(self):
        return "Manager"

class Intern2(Employee2):
    def role(self):
        return "Intern"

class EmployeeFactory2:
    registry = {
        # TODO: map strings to classes, e.g. "developer": Developer2
    }

    @classmethod
    def create(cls, employee_type, name):
        # TODO: normalize input, lookup in registry, return instance
        pass

class Company2:
    def __init__(self, name):
        self.name = name
        self._employees = []

    def add(self, employee):
        # TODO: validate type
        pass

    def list_by_role(self, role):
        # TODO
        pass

    def headcount_by_role(self):
        # TODO
        pass

# Demo (after implementation):
# c = Company2("ACME")
# c.add(EmployeeFactory2.create("developer", "Alice Smith"))
# c.add(EmployeeFactory2.create("manager", "Bob Stone"))
# c.add(EmployeeFactory2.create("intern", "Charlie Young"))
# print(c.headcount_by_role())
# print([e.email for e in c.list_by_role("Developer")])

## Exercise 2 — ABC interface: Notifier

Implement a base class `Notifier` with:
- `send(to, message)` abstract method

Create:
- `ConsoleNotifier` prints messages
- `ListNotifier` stores messages for tests

Then implement `welcome_user(notifier, username)`.

In [ ]:
# Exercise 2 starter

from abc import ABC, abstractmethod

class Notifier(ABC):
    @abstractmethod
    def send(self, to, message):
        pass

class ConsoleNotifier(Notifier):
    def send(self, to, message):
        # TODO
        pass

class ListNotifier(Notifier):
    def __init__(self):
        self.sent = []

    def send(self, to, message):
        # TODO
        pass

def welcome_user(notifier, username):
    # TODO: notifier.send(username, f"Welcome {username}!")
    pass

# Demo:
# n1 = ConsoleNotifier()
# welcome_user(n1, "alice")
# n2 = ListNotifier()
# welcome_user(n2, "bob")
# print(n2.sent)

## Exercise 3 — Composition: Exporters for reports

Create `Report` with an exporter object.
Implement:
- `TextExporter`
- `JSONExporter`

Then show that swapping exporters changes output without changing Report logic.

In [ ]:
# Exercise 3 starter

class Report2:
    def __init__(self, title, data, exporter):
        self.title = title
        self.data = data
        self.exporter = exporter

    def export(self):
        return self.exporter.export(self.title, self.data)

class TextExporter2:
    def export(self, title, data):
        # TODO
        pass

class JSONExporter2:
    def export(self, title, data):
        # TODO
        pass

# Demo:
# r = Report2("Weekly", {"sales": 10, "profit": 3}, TextExporter2())
# print(r.export())
# r.exporter = JSONExporter2()
# print(r.export())

---

## Checklist for Unit 8

- [ ] Encapsulation (`_attr`, `__attr`) and why it matters
- [ ] Properties for controlled access
- [ ] Abstract base classes for contracts
- [ ] `@classmethod` vs `@staticmethod`
- [ ] Composition vs inheritance (design choice)
- [ ] Factory pattern (registry-based)
- [ ] Singleton pattern (and trade-offs)

Next: **Unit 9 — Computational Thinking & Algorithms**